# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

Submit "ratings.tsv" in Gradescope.

In [9]:
import pandas as pd
import random

# Load the recipes.tsv file
recipes = pd.read_csv("recipes.tsv", sep="\t")


def assign_raw_rating(row):
    title = row['recipe_title'].lower()
    intro = row['recipe_introduction'].lower()

    # High comfort foods
    if any(word in title for word in ['bacon', 'fried', 'brownie', 'crisp', 'cookie', 'mac', 'cheese', 'chicken', 'ramen', 'burger', 'lasagna']):
        return 5
    elif any(word in title for word in ['soup', 'bolognese', 'burrito', 'pasta', 'muffin']):
        return 4
    elif any(word in title for word in ['scallop', 'sujebi', 'souffle']):
        return 3
    elif any(word in title for word in ['primavera', 'croissant', 'salad', 'quinoa']):
        return 1
    else:
        return random.randint(2, 4)

# Assign raw ratings
recipes['rating_raw'] = recipes.apply(assign_raw_rating, axis=1)


comfort_keywords = ['bacon', 'fried', 'brownie', 'cookie', 'crisp', 'burger', 'ramen', 'chicken']
comfort_matches = recipes[recipes['recipe_title'].str.lower().str.contains('|'.join(comfort_keywords))]

# Set top 10 comfort recipes to raw rating 5
recipes.loc[comfort_matches.head(10).index, 'rating_raw'] = 5

# Set least comfort foods to raw rating 1
non_comfort_keywords = ['primavera', 'croissant', 'souffle', 'salad', 'quinoa']
non_comfort_matches = recipes[recipes['recipe_title'].str.lower().str.contains('|'.join(non_comfort_keywords))]

recipes.loc[non_comfort_matches.head(10).index, 'rating_raw'] = 1

# Map raw ratings (1-5) to 0.0–1.0 scale
recipes['rating'] = (recipes['rating_raw'] - 1) * 0.25

# Select required columns
ratings = recipes[['recipe_slug', 'rating']]

# Save to ratings.tsv
ratings.to_csv("ratings.tsv", sep="\t", index=False)

print("ratings.tsv created with comfort food ratings.")


ratings.tsv created with comfort food ratings.


## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [10]:
import pandas as pd

tags = pd.read_csv("recipe-tags.tsv", sep="\t")
print(tags.columns)

Index(['recipe_slug', 'recipe_tag'], dtype='object')


In [11]:
import pandas as pd

# Load ratings.tsv
ratings = pd.read_csv("ratings.tsv", sep="\t")

# Load recipe-tags.tsv
tags = pd.read_csv("recipe-tags.tsv", sep="\t")

# One-hot encode the recipe_tag column
tag_features = tags.assign(value=1).pivot_table(
    index='recipe_slug',
    columns='recipe_tag',
    values='value',
    fill_value=0
).reset_index()

# Add bias column (hardcoded to 1)
tag_features['bias'] = 1

# Reorder columns: recipe_slug, bias, then all tags (sorted for consistency)
cols = ['recipe_slug', 'bias'] + sorted(
    [col for col in tag_features.columns if col not in ['recipe_slug', 'bias']]
)
tag_features = tag_features[cols]

# Merge with ratings to ensure same order
features = ratings[['recipe_slug']].merge(tag_features, on='recipe_slug', how='left')

# Fill any missing values (in case some recipes had no tags)
features = features.fillna(0)

# Save to TSV
features.to_csv("features.tsv", sep="\t", index=False)

print("features.tsv created successfully with recipe_tag one-hot encoding.")


features.tsv created successfully with recipe_tag one-hot encoding.


Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [12]:
# YOUR CHANGES HERE

import pandas as pd
from sklearn.linear_model import Ridge

# Load
features = pd.read_csv("features.tsv", sep="\t")
ratings = pd.read_csv("ratings.tsv", sep="\t")


X = features.drop(columns=['recipe_slug'])  # Input features
y = ratings['rating']                       # Target ratings

# Fit 
model = Ridge(alpha=1.0, fit_intercept=False)
model.fit(X, y)


,alpha,1.0
,fit_intercept,False
,copy_X,True
,max_iter,None
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,None


Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not add anything for the `intercept_` attribute of a scikit-learn model; this will be covered by the coefficient for the bias column added in part 2.

In [13]:
# YOUR CHANGES HERE

coefs = pd.DataFrame({
    'recipe_tag': X.columns,
    'coefficient': model.coef_
})

# Save to model.tsv
coefs.to_csv("model.tsv", sep="\t", index=False)

print("model.tsv created with ridge regression coefficients.")


model.tsv created with ridge regression coefficients.


Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [14]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np


features = pd.read_csv("features.tsv", sep="\t")
model_coefs = pd.read_csv("model.tsv", sep="\t")

# Extract feature matrix 
X = features.drop(columns=['recipe_slug'])

# Align model coefficients to feature columns
coef_dict = dict(zip(model_coefs['recipe_tag'], model_coefs['coefficient']))
coefs_aligned = np.array([coef_dict[col] for col in X.columns])


score_estimate = X.values.dot(coefs_aligned)


estimates = pd.DataFrame({
    'recipe_slug': features['recipe_slug'],
    'score_estimate': score_estimate
})

# Save to TSV
estimates.to_csv("estimates.tsv", sep="\t", index=False)

print("estimates.tsv created with predicted recipe scores.")


estimates.tsv created with predicted recipe scores.


Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [17]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

# Parameters
lambda_reg = 1.0  # Ridge regression regularization
alpha = 2.0       # LinUCB exploration parameter

# Load data
features = pd.read_csv("features.tsv", sep="\t")
ratings = pd.read_csv("ratings.tsv", sep="\t")

#  matrices
X = features.drop(columns=['recipe_slug']).values  # Feature matrix (n x d)
y = ratings['rating'].values                        # Target ratings (n,)

# Calculate A = X^T X + lambda * I
d = X.shape[1]
A = X.T @ X + lambda_reg * np.eye(d)

# Invert A
A_inv = np.linalg.inv(A)

# Calculate ridge regression coefficients beta_hat
beta_hat = A_inv @ (X.T @ y)

# Calculate mean estimates for each recipe
mu = X @ beta_hat  # shape (n,)

# Calculate confidence widths for each recipe
confidence_widths = np.sqrt(np.sum(X @ A_inv * X, axis=1))

# Calculate upper confidence bounds
score_bounds = mu + alpha * confidence_widths

# Prepare output DataFrame
bounds = pd.DataFrame({
    'recipe_slug': features['recipe_slug'],
    'score_bound': score_bounds
})

# Save to TSV
bounds.to_csv("bounds.tsv", sep="\t", index=False)

print("bounds.tsv created with LinUCB upper confidence bounds.")


bounds.tsv created with LinUCB upper confidence bounds.


Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

In [16]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

# Parameters
lambda_reg = 1.0
alpha = 2.0

# Load data
features = pd.read_csv("features.tsv", sep="\t")
ratings = pd.read_csv("ratings.tsv", sep="\t")

# Map from recipe_slug to index for quick lookup
slug_to_idx = {slug: i for i, slug in enumerate(features['recipe_slug'])}

# Feature matrix
X = features.drop(columns=['recipe_slug']).values
n_recipes, d = X.shape

# Ratings vector aligned with features
y = ratings.set_index('recipe_slug').loc[features['recipe_slug'], 'rating'].values

# Initialize LinUCB parameters
A = lambda_reg * np.eye(d)    # d x d matrix
b = np.zeros(d)               # d-vector

# Keep track of which recipes have been recommended
recommended = set()

# Store recommendations info
recommendations = []

for step in range(n_recipes):
    # Compute A_inv
    A_inv = np.linalg.inv(A)
    
    # Calculate beta_hat = A_inv @ b
    beta_hat = A_inv @ b
    
    # This is to compute upper confidence bounds for all the unrecommended recipes
    bounds = []
    for i in range(n_recipes):
        if i in recommended:
            bounds.append(-np.inf)  # Ignore already recommended
            continue
        x_i = X[i]
        mu = x_i @ beta_hat
        conf = alpha * np.sqrt(x_i @ A_inv @ x_i)
        bounds.append(mu + conf)
    
    
    chosen_idx = np.argmax(bounds)
    chosen_slug = features.loc[chosen_idx, 'recipe_slug']
    chosen_bound = bounds[chosen_idx]
    reward = y[chosen_idx]
    
    
    recommendations.append({
        'recipe_slug': chosen_slug,
        'score_bound': chosen_bound,
        'reward': reward
    })
    
    
    x_chosen = X[chosen_idx]
    A += np.outer(x_chosen, x_chosen)
    b += reward * x_chosen
    
    
    recommended.add(chosen_idx)

# Save recs
recommendations_df = pd.DataFrame(recommendations)
recommendations_df.to_csv("recommendations.tsv", sep="\t", index=False)

print("recommendations.tsv created with LinUCB online recommendations.")


recommendations.tsv created with LinUCB online recommendations.


Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

In [18]:
content = """I read the readings provided in Week 4 resources. I also used ChatGPT for assistance with understanding with Python code examples.
"""

with open("acknowledgments.txt", "w") as f:
    f.write(content)

print("acknowledgments.txt created")

acknowledgments.txt created


## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.